# YOLO from Scratch 

Some nuances: 
* Last layer will  have one filter corresponding to each bounding box prediction. 
Eg. Below, We have 7*7 grids. 2 anchor boxes for each of them.20 classes.  

So,   

no of classes {class score} + /[(objectness score {whether object or background} + 4 coordinates {if objeect, else dont care})* no of classes/]*no. of boxes 

For each cell. We have a matrix of 7*7 with the above depth.

![](helper/1.JPG)

* Out of the two boxes, we find the box which matches better and reduce the confidence for the other box. So we can only have one class prediction per grid cell (limitation). Hence, in the above calculation, we only one P(class_i|object) rather than two for two boxes. 
We need to find which box out of the two to choose. According to the paper, the box with higher IOU is responsible for prediction and only that appears in the loss 
![](helper/2.JPG)
![](helper/3.JPG)


Important points to note: 
* Loss function is sum of squared errors. This gives equal weightage to classification and localization error, which may not be desirable. So we give weightage to localization error ($\lambda$ = 5). 
* Also, there usually exist lot amny background boxes which may have zero loss and we get a high representation of cells with objects. So we give less weightage to loss from non object boxes to ensure training stability. (not very clear)
* We want localization error in smaller objects to be weighted more than same error in bigger objects. Hence we take square of difference of root of width and height. 
eg. 
1. gt = 100, pred = 94. mse = 1024, yolo loss component = 0.09 #(100**(1/2)-94**(1/2))**2
2. gt = 8, pred = 2, mse = 36, yolo = 2. #(8**(1/2)-2**(1/2))**2
We yolo trick penalizes localization error more in smaller boxes (for same magnitude variation in big and small boxes).

In [ ]:
    label_matrix = np.zeros([7, 7, 30])
    for l in label:
        l = l.split(',')
        l = np.array(l, dtype=np.int)
        xmin = l[0]
        ymin = l[1]
        xmax = l[2]
        ymax = l[3]
        cls = l[4]

        ## Convert coordinates to [0,1]
        x = (xmin + xmax) / 2 / image_w
        y = (ymin + ymax) / 2 / image_h
        w = (xmax - xmin) / image_w
        h = (ymax - ymin) / image_h

        # Since output layer has 7*7 grid, this gives us index of the grid responsible for detection. 
        loc = [7 * x, 7 * y]
        loc_i = int(loc[1])
        loc_j = int(loc[0])

        ## x, y are offset inside that grid cell.
        y = loc[1] - loc_i
        x = loc[0] - loc_j

        #We need to store GT for only the grid responsible (mid point of GT lies in that grid)
        # Does the below code store the labels in box1, keeping box2 empty? Doubt

        if label_matrix[loc_i, loc_j, 24] == 0:
            label_matrix[loc_i, loc_j, cls] = 1 # index 0 to 20 are conditional class probabilities 
            label_matrix[loc_i, loc_j, 20:24] = [x, y, w, h]
            label_matrix[loc_i, loc_j, 24] = 1  # response

*Reference*

(slide 28) from https://docs.google.com/presentation/d/1kAa7NOamBt4calBU9iHgT8a86RRHz9Yz2oh4-GTdX6M/edit#slide=id.p

output and label relative to the cell, x,y,w,h. x,y belongs to [0,1], w,h can be greater thn 1 if object is larger than that cell. 
![](helper/5.JPG)
![](helper/6.JPG)

Target shape for one image = S*S*25  
Prediction shape for one image = S*S*30 (Target/GT has one label bb, while for prediction we are predicting two boxes per cell )

Doubts: 
how pc ensured to be between 0,1 in prediction?
How exactly backpropagation happens from 7*7*30 layer to fully connected layer?
What if multiple objects have box in on cell? Any way to detect that and finegrain the grid? Or we will just train for the last object updated?
